##P2: Transfer Learning and Feature Extraction

**1) General imports and definitions**

In [1]:
#Check if NVIDIA GPU is enabled
!nvidia-smi

# IMPORTS
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import skimage.io as sio
import tensorflow as tf
import keras

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.models import Sequential, model_from_json, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

print( 'Using Keras version', keras.__version__)

from google.colab import drive
drive.mount('/content/gdrive')
#!ln -s /content/gdrive/My\ Drive/ /mydrive

Mon May 23 15:57:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**2) Extract images plus create training, validation and test datasets**

In [2]:
!mkdir data
# 
!cp /content/gdrive/MyDrive/DL/L2/MAMe_data_256.zip .
!cp /content/gdrive/MyDrive/DL/L2/MAMe_metadata.zip .
#!unzip /content/gdrive/MyDrive/DL/L2/MAMe_data_256.zip -d content/data
!unzip  -qq MAMe_data_256.zip -d data

In [3]:
!unzip MAMe_metadata.zip
!cp /content/gdrive/MyDrive/DL/L2/MAMe_dataset.csv .


Archive:  MAMe_metadata.zip
  inflating: MAMe_dataset.csv        
  inflating: MAMe_labels.csv         
  inflating: MAMe_toy_dataset.csv    


In [4]:
labels_list = pd.read_csv("/content/MAMe_dataset.csv")
labels = pd.read_csv("/content/MAMe_labels.csv", names=["id", "name"])

final_labels = labels["name"].tolist()
labels_list["Medium"] = labels_list["Medium"].apply(final_labels.index)

# Training dataset
train_labels_list = labels_list[labels_list["Subset"] == "train"]

# Validation dataset
val_labels_list = labels_list[labels_list["Subset"] == "val"]

# Test dataset
test_labels_list = labels_list[labels_list["Subset"] == "test"]



In [5]:
from skimage.transform import rotate
import random
labels = np.arange(29)
img_rows, img_cols, channels = 256, 256, 3
input_shape = (img_rows, img_cols, channels)

class CustomDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, set_labels_list, batch_size, shuffle=True):
        'Initialization'
        self.set_labels_list = set_labels_list
        self.batch_size = batch_size
        self.shuffle = shuffle

        self.all_y = np.array(self.set_labels_list["Medium"])
        self.all_y = np_utils.to_categorical(self.all_y, 29)

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.set_labels_list['Image file']) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        X = []
        for train_file in self.set_labels_list["Image file"][index*self.batch_size:(index*self.batch_size)+self.batch_size]:
            im = sio.imread("data/data_256/" + str(train_file))
            #num = random.randint(0,3)
            #if num % 2 == 0:
            #  im = rotate(im, random.randint(0,350), resize=False, center=None, order=None)
            #im = resize(im, (im.shape[0] // 4 , im.shape[1] // 4), anti_aliasing=True)
            X.append(im)  
        X = np.array(X) / 255.0
        X = X.reshape(X.shape[0], img_rows, img_cols, channels)
        y = self.all_y[index*self.batch_size:(index*self.batch_size)+self.batch_size]
        return X, y

In [6]:
# Hyperparameters
batch_norm = False
dropout = False
double_conv = False
learning_rate = 0.001
epochs = 100
batch_size = 128
hidden_neurons = 32

## Model

In [ ]:
from keras.layers.advanced_activations import PReLU

model = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_shape = input_shape)

for layer in model.layers[:12]:
  layer.trainable = False

# Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(4096, activation=PReLU())(x)
x = Dense(1024, activation=PReLU())(x)
predictions = Dense(29, activation="softmax")(x)

# creating the final model 
model_final = Model(model.input, predictions)
print(model_final.summary())

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0   

In [ ]:
train = CustomDataGenerator(train_labels_list, batch_size=batch_size, shuffle=True)
val = CustomDataGenerator(val_labels_list, batch_size=batch_size, shuffle=True)
test = CustomDataGenerator(test_labels_list, batch_size=batch_size, shuffle=True)

datagen = ImageDataGenerator(
        rotation_range=0,  
        zoom_range = 0.0,  
        width_shift_range=0.1, 
        height_shift_range=0.1,
        horizontal_flip=False)

reduce_lr= ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.1e-6)

"""model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    '/mydrive/2022/DL/P1/model',
    monitor="val_loss",
    save_best_only=True,
)"""

early = EarlyStopping(
    patience = 5,
    min_delta = 1e-3,
    restore_best_weights=True)

## Training

In [ ]:
# Compile the NN
model_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

# Start training
history = model_final.fit(train, validation_data=val, epochs = epochs, callbacks=[early, reduce_lr], batch_size = 128)

Epoch 1/100
158/158 [==============================] - 85s 528ms/step - loss: 3.5615 - accuracy: 0.0435 - val_loss: 3.2992 - val_accuracy: 0.0455 - lr: 0.0010
Epoch 2/100
158/158 [==============================] - 83s 524ms/step - loss: 3.1873 - accuracy: 0.0909 - val_loss: 2.9219 - val_accuracy: 0.1349 - lr: 0.0010
Epoch 3/100
158/158 [==============================] - 83s 523ms/step - loss: 2.8356 - accuracy: 0.1721 - val_loss: 2.6132 - val_accuracy: 0.2791 - lr: 0.0010
Epoch 4/100
158/158 [==============================] - 83s 525ms/step - loss: 2.6137 - accuracy: 0.2197 - val_loss: 2.3038 - val_accuracy: 0.2784 - lr: 0.0010
Epoch 5/100
158/158 [==============================] - 83s 523ms/step - loss: 2.2871 - accuracy: 0.2935 - val_loss: 1.8594 - val_accuracy: 0.4134 - lr: 0.0010
Epoch 6/100
158/158 [==============================] - 83s 523ms/step - loss: 1.9549 - accuracy: 0.3788 - val_loss: 1.6411 - val_accuracy: 0.4787 - lr: 0.0010
Epoch 7/100
158/158 [=========================

## Evaluation

In [ ]:
# Evaluate the model with validation set
test = CustomDataGenerator(test_labels_list, batch_size=batch_size, shuffle=True)

score = model_final.evaluate(test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9076278805732727
Test accuracy: 0.7453893423080444


In [ ]:
# Store Plots
matplotlib.use('Agg')

# Accuracy plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('cnn_accuracy.pdf')
plt.close()

# Loss plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.savefig('cnn_loss.pdf')
plt.close()

**7) Saving model and weights**

In [ ]:
# Saving model and weights
model_json = model_final.to_json()
with open('model.json', 'w') as json_file:
        json_file.write(model_json)
weights_file = "weights-MAMe_"+str(score[1])+".hdf5"
model.save_weights(weights_file,overwrite=True)

## Feature Extraction

In [7]:
def full_network_embedding(model, image_paths, batch_size, target_layer_names, input_reshape, stats=None):
  feature_extractor = tf.keras.Model(
        inputs=model.inputs,
        outputs=[layer.output for layer in model.layers if layer.name in target_layer_names],
    )
  get_raw_features = lambda x: [tensor.numpy() for tensor in feature_extractor(x)]

  # Prepare output variable
  feature_shapes = [layer.output_shape for layer in model.layers if layer.name in target_layer_names]
  len_features = sum(shape[-1] for shape in feature_shapes)
  features = np.empty((len(image_paths), len_features))

  # Extract features
  for idx in range(0, len(image_paths), batch_size):
      batch_images_path = image_paths[idx:idx + batch_size]
      img_batch = np.zeros((len(batch_images_path), *input_reshape, 3), dtype=np.float32)
      for i, img_path in enumerate(batch_images_path):
          cv_img = cv2.imread(img_path)
          try:
              cv_img_resize = cv2.resize(cv_img, input_reshape)
              img_batch[i] = np.asarray(cv_img_resize, dtype=np.float32)[:, :, ::-1]
          except:
              print(img_path)

      feature_vals = get_raw_features(img_batch)
      features_current = np.empty((len(batch_images_path), 0))
      for feat in feature_vals:
          #If its not a conv layer, add without pooling
          if len(feat.shape) != 4:
              features_current = np.concatenate((features_current, feat), axis=1)
              continue
          #If its a conv layer, do SPATIAL AVERAGE POOLING
          pooled_vals = np.mean(np.mean(feat, axis=2), axis=1)
          features_current = np.concatenate((features_current, pooled_vals), axis=1)
      # Store in position
      features[idx:idx+len(batch_images_path)] = features_current.copy()

  # STANDARDIZATION STEP
  # Compute statistics if needed
  if stats is None:
      stats = np.zeros((2, len_features))
      stats[0, :] = np.mean(features, axis=0)
      stats[1, :] = np.std(features, axis=0)
  # Apply statistics, avoiding nans after division by zero
  features = np.divide(features - stats[0], stats[1], out=np.zeros_like(features), where=stats[1] != 0)
  if len(np.argwhere(np.isnan(features))) != 0:
      raise Exception('There are nan values after standardization!')
  # DISCRETIZATION STEP
  th_pos = 0.15
  th_neg = -0.25
  features[features > th_pos] = 1
  features[features < th_neg] = -1
  features[[(features >= th_neg) & (features <= th_pos)][0]] = 0

  # # Store output
  import os
  outputs_path = '.'
  if not os.path.exists(outputs_path):
    os.makedirs(outputs_path)
  np.save(os.path.join(outputs_path, 'fne.npy'), features)
  np.save(os.path.join(outputs_path, 'stats.npy'), stats)

  # Return
  return features, stats

In [8]:
initial_model = applications.vgg16.VGG16(weights="imagenet", include_top=True,
                                                input_shape=(224, 224, 3))
target_layer_names = ['block1_conv1', 'block1_conv2', 'block2_conv1', 'block2_conv2', 'block3_conv1', 'block3_conv2',
                          'block3_conv3', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block5_conv1', 'block5_conv2',
                          'block5_conv3']

y_train = np.array(train_labels_list["Medium"])
y_train = np_utils.to_categorical(y_train, 29)
y_test = np.array(test_labels_list["Medium"])
y_test = np_utils.to_categorical(y_train, 29)


553476096/553467096 [==============================] - 3s 0us/step


In [9]:
## getting images
import cv2
train_images, test_images = [], []
for image in train_labels_list['Image file']:
  train_images.append('/content/data/data_256/'+ image)
print('Total train images:', len(train_images), ' with their corresponding', len(y_train), 'labels')

for image in test_labels_list['Image file']:
  test_images.append('/content/data/data_256/'+ image)
print('Total train images:', len(train_images), ' with their corresponding', len(y_test), 'labels')


Total train images: 20300  with their corresponding 20300 labels
Total train images: 20300  with their corresponding 20300 labels


In [13]:
fne_features, fne_stats_train = full_network_embedding(initial_model, train_images, 16 ,target_layer_names, (224,224))
print(fne_features.shape, "\n", fne_stats_train)

(20300, 4224) 
 [[21.59794746 62.87631663 69.53910667 ...  0.3470342   0.32718058
   0.48778457]
 [10.2440627  28.74682553 21.05250792 ...  0.92889994  0.74857569
   1.45328083]]


In [14]:
from sklearn import svm
"""
# Train SVM with the obtained features.
clf = svm.LinearSVC()
clf.fit(X=fne_features, y=train_labels_list["Medium"])
print('Done training SVM on extracted features of training set')

# Test SVM with the test set.
predicted_labels = clf.predict(fne_features)
print('Done testing SVM on extracted features of test set')
"""
# Train SVM with the obtained features.
clf = svm.LinearSVC()
clf.fit(X=fne_features, y=train_labels_list["Medium"])
print('Done training SVM on extracted features of training set')
del fne_features

# Call FNE method on the test set, using stats from training
fne_features, fne_stats_train = full_network_embedding(initial_model, test_images, 16 ,
                                                        target_layer_names, (224,224), stats=fne_stats_train)
print('Done extracting features of test set')

# Test SVM with the test set.
predicted_labels = clf.predict(fne_features)
print('Done testing SVM on extracted features of test set')
del fne_features

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


Done training SVM on extracted features of training set
Done extracting features of test set
Done testing SVM on extracted features of test set


### Classification report FE

In [15]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_labels_list["Medium"], predicted_labels))
print("-------------------------------------------------------------------")
print(confusion_matrix(test_labels_list["Medium"], predicted_labels))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       700
           1       0.73      0.67      0.70       700
           2       0.77      0.74      0.75       700
           3       0.78      0.83      0.81       313
           4       0.71      0.73      0.72       700
           5       0.73      0.66      0.70       700
           6       0.83      0.81      0.82       700
           7       0.79      0.72      0.76       700
           8       0.83      0.86      0.85       700
           9       0.69      0.93      0.79       188
          10       0.93      0.99      0.96       328
          11       0.93      0.96      0.95       584
          12       0.63      0.80      0.71       265
          13       0.68      0.64      0.66       572
          14       0.75      0.69      0.72       700
          15       0.83      0.71      0.76       700
          16       0.52      0.73      0.61       257
          17       0.81    